# Log, Trace, and Monitor | ログ、トレース、監視Langchainを使用してアプリやエージェントを構築する際、単一のユーザーリクエストを満たすためには複数のAPIコールが必要になります。しかし、これらのリクエストは分析する際には連携されていません。[**Portkey**](/docs/ecosystem/integrations/portkey)を利用すると、単一のユーザーリクエストに関連する全ての埋め込み、完了、その他のリクエストがログに記録され、共通のIDで追跡されます。これにより、ユーザーのインタラクションを完全に可視化することが可能になります。> When building apps or agents using Langchain, you end up making multiple API calls to fulfill a single user request. However, these requests are not chained when you want to analyse them. With [**Portkey**](/docs/ecosystem/integrations/portkey), all the embeddings, completion, and other requests from a single user request will get logged and traced to a common ID, enabling you to gain full visibility of user interactions.このノートブックは、Langchainアプリで`Portkey`を使用してLangchain LLMの呼び出しをログに記録し、追跡し、監視する方法についてのステップバイステップガイドとして機能します。> This notebook serves as a step-by-step guide on how to log, trace, and monitor Langchain LLM calls using `Portkey` in your Langchain app.

まず、Portkey、OpenAI、そしてAgentツールをインポートしましょう> First, let's import Portkey, OpenAI, and Agent tools

In [ ]:
import os

from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI
from langchain.utilities import Portkey

以下にあなたのOpenAI APIキーを貼り付けてください。[(こちらで見つけることができます)](https://platform.openai.com/account/api-keys)> Paste your OpenAI API key below. [(You can find it here)](https://platform.openai.com/account/api-keys)

In [ ]:
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

## Get Portkey API Key | Portkey APIキーを取得する1. [こちらからPortkeyに登録してください](https://app.portkey.ai/login)   > Sign up for [Portkey here](https://app.portkey.ai/login)2. [ダッシュボード](https://app.portkey.ai/)で、左上にあるプロフィールアイコンをクリックし、その後「APIキーをコピー」をクリックしてください。   > On your [dashboard](https://app.portkey.ai/), click on the profile icon on the top left, then click on "Copy API Key"3. 以下に貼り付けてください   > Paste it below

In [ ]:
PORTKEY_API_KEY = "<PORTKEY_API_KEY>"  # Paste your Portkey API Key here

## Set Trace ID | トレースIDを設定1. 以下にあなたのリクエストのトレースIDを設定してください   > Set the trace id for your request below2. 単一のリクエストから発生するすべてのAPIコールに対して、Trace IDが共通であることがあります   > The Trace ID can be common for all API calls originating from a single request

In [ ]:
TRACE_ID = "portkey_langchain_demo"  # Set trace id here

## Generate Portkey Headers | Portkey ヘッダーを生成する

In [ ]:
headers = Portkey.Config(
    api_key=PORTKEY_API_KEY,
    trace_id=TRACE_ID,
)

エージェントを通常通り実行してください。**唯一の変更点**は、これからリクエストに**上記のヘッダーを含める**ことです。> Run your agent as usual. The **only** change is that we will **include the above headers** in the request now.

In [ ]:
llm = OpenAI(temperature=0, headers=headers)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# Let's test it out!
agent.run(
    "What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?"
)

## How Logging & Tracing Works on Portkey | Portkeyでのログ記録とトレースの仕組み**ログ記録**> **Logging*** Portkeyを通じてリクエストを送信することで、デフォルトで全てのリクエストが記録されます  > Sending your request through Portkey ensures that all of the requests are logged by default* 各リクエストログには、`timestamp`（タイムスタンプ）、`model name`（モデル名）、`total cost`（総コスト）、`request time`（リクエスト時間）、`request json`（リクエストJSON）、`response json`（レスポンスJSON）、および追加のPortkey機能が含まれています  > Each request log contains `timestamp`, `model name`, `total cost`, `request time`, `request json`, `response json`, and additional Portkey features**トレーシング**> **Tracing*** トレースIDは各リクエストに添付され、Portkeyダッシュボードのログに表示されます  > Trace id is passed along with each request and is visibe on the logs on Portkey dashboard* また、必要であれば、各リクエストに**固有のトレースID**を設定することもできます  > You can also set a **distinct trace id** for each request if you want* ユーザーフィードバックをトレースIDにも追加することができます。[この件の詳細はこちら](https://docs.portkey.ai/key-features/feedback-api)  > You can append user feedback to a trace id as well. [More info on this here](https://docs.portkey.ai/key-features/feedback-api)

## Advanced LLMOps Features - Caching, Tagging, Retries | 高度なLLMOpsの機能 - キャッシング、タグ付け、リトライログ記録やトレーシングに加えて、Portkeyは既存のワークフローに生産機能を追加する追加の機能も提供します：> In addition to logging and tracing, Portkey provides more features that add production capabilities to your existing workflows:**キャッシング**> **Caching**以前に対応した顧客の問い合わせについては、OpenAIに再度送信する代わりにキャッシュから応答します。完全に一致する文字列または意味的に類似した文字列にマッチします。キャッシュを利用することでコストを削減し、レイテンシーを20倍短縮できます。> Respond to previously served customers queries from cache instead of sending them again to OpenAI. Match exact strings OR semantically similar strings. Cache can save costs and reduce latencies by 20x.**リトライ**> **Retries**失敗したAPIリクエストを自動的に**最大5回**再処理します。ネットワークの過負荷を防ぐために、リトライ試行間の間隔を広げる**指数バックオフ**戦略を使用します。> Automatically reprocess any unsuccessful API requests **`upto 5`** times. Uses an **`exponential backoff`** strategy, which spaces out retry attempts to prevent network overload.| 機能 | 設定キー | 値（タイプ） || -- | -- | -- || [🔁 自動リトライ](https://docs.portkey.ai/key-features/automatic-retries) | `retry_count` | `integer` \[1,2,3,4,5] || [🧠 キャッシュを有効にする](https://docs.portkey.ai/key-features/request-caching) | `cache` | `simple` または `semantic` |> | Feature | Config Key | Value (Type) |> | -- | -- | -- |> | [🔁 Automatic Retries](https://docs.portkey.ai/key-features/automatic-retries) | `retry_count` | `integer` \[1,2,3,4,5] |> | [🧠 Enabling Cache](https://docs.portkey.ai/key-features/request-caching) | `cache` | `simple` OR `semantic` |**タグ付け**> **Tagging**事前に定義されたタグを使用して、各ユーザーのインタラクションを高い詳細度で追跡および監査します。> Track and audit ach user interaction in high detail with predefined tags.| タグ | 設定キー | 値（タイプ） || -- | -- | -- || ユーザータグ | `user` | `string` || 組織タグ | `organisation` | `string` || 環境タグ | `environment` | `string` || プロンプトタグ（バージョン/ID/文字列） | `prompt` | `string` |> | Tag | Config Key | Value (Type) |> | -- | -- | -- |> | User Tag | `user` | `string` |> | Organisation Tag | `organisation` | `string` |> | Environment Tag | `environment` | `string` |> | Prompt Tag (version/id/string) | `prompt` | `string` |

## Code Example With All Features | すべての機能を含むコード例

In [ ]:
headers = Portkey.Config(
    # Mandatory
    api_key="<PORTKEY_API_KEY>",
    # Cache Options
    cache="semantic",
    cache_force_refresh="True",
    cache_age=1729,
    # Advanced
    retry_count=5,
    trace_id="langchain_agent",
    # Metadata
    environment="production",
    user="john",
    organisation="acme",
    prompt="Frost",
)

llm = OpenAI(temperature=0.9, headers=headers)

print(llm("Two roads diverged in the yellow woods"))